In [1]:
MAX_ROUNDS = 400
OPTIMIZE_ROUNDS = False
LEARNING_RATE = 0.07
EARLY_STOPPING_ROUNDS = 50  

# 참고: EARRY_STOPPING_ROUNDS를 높게 설정하여 (Optimize_ROUNDS가 설정된 경우)
# 나는 많은 정보를 얻어 스스로 판단할 것입니다. 당신은 아마
# 실제 얼리 스톱을 하려면 EARRY_STOPPING_ROUNDS를 줄입니다

처음에 MAX_ROUNDS를 상당히 높게 설정하고 Optimized_ROUNDS를 사용하여 적절한 라운드 수를 파악하는 것이 좋습니다. (모든 폴드 중 best_ntree_limit의 최대값에 가까워야 한다고 생각합니다. 모델이 적절하게 정규화되어 있다면 더 높을 수도 있습니다.) 또는 verbose=True를 설정하고 세부 정보를 살펴 모든 폴드에 적합한 라운드 수를 찾을 수 있습니다. 그런 다음 Optimized_ROUNDS를 끄고 MAX_ROUNDS를 적절한 총 라운드 수로 설정합니다.

각 폴드에 가장 적합한 라운드를 선택하여 "조기 정지"를 하는 경우의 문제점은 검증 데이터에 과도하게 적합하다는 것입니다. 따라서 테스트 데이터를 예측하기 위한 최적의 모델을 생성하지 못할 가능성이 높으며, 테스트 데이터를 다른 모델과 적층/조립하기 위한 검증 데이터를 생성하는 데 사용하면 테스트 데이터가 앙상블에서 너무 많은 비중을 차지하게 됩니다. 또 다른 가능성(XGBoost의 기본값인 경우)은 최상의 라운드가 아닌 조기 정지가 실제로 발생하는 라운드를 사용하는 것입니다. (만약 지연이 충분히 길다면) 과적합 문제는 해결되지만, 지금까지는 도움이 되지 않은 것 같습니다. (모든 폴드에 대해 일정한 라운드 수를 사용하는 것보다 폴드당 20라운드를 조기 정지하는 것이 더 나쁜 검증 점수를 얻었으므로 조기 정지는 실제로 과소 적합된 것 같습니다.)

In [2]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from numba import jit
import time
import gc

In [3]:
# Compute gini

# from CPMP's kernel https://www.kaggle.com/cpmpml/extremely-fast-gini-computation
@jit
def eval_gini(y_true, y_prob):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    return gini

In [7]:
def gini_xgb(preds, dtrain):
    labels = dtrain.get_label()
    gini_score = -eval_gini(labels, preds)
    return [('gini', gini_score)]


def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))


def target_encode(trn_series=None,    # Revised to encode validation series
                  val_series=None,
                  tst_series=None,
                  target=None,
                  min_samples_leaf=1,
                  smoothing=1,
                  noise_level=0):
    
    """
    평활화는 Danielle Micci-Barreca에 의해 다음 논문과 같이 계산됩니다
https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf
trn_series : pd.Series로 범주형 기능 교육
tst_series : pd.Series로 범주형 피쳐 테스트
대상: 대상 데이터를 pd.Series로 사용
min_ samples_leaf (int) : 범주 평균을 고려한 최소 표본
smoothing (int) : smoothing effect to balance categorical average vs prior
    """
    
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    
    # 계산대상평균
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    
    # 모든 대상 데이터에 평균 함수 적용
    prior = target.mean()
    
    # 카운트가 클수록 full_avg가 덜 고려
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    
    # 평균을 trn 및 tst 영상 시리즈에 적용
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    
    # pd.merge는 인덱스를 유지하지 않으므로 복원
    ft_trn_series.index = trn_series.index
    ft_val_series = pd.merge(
        val_series.to_frame(val_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=val_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    
    # pd.merge는 인덱스를 유지하지 않으므로 복원
    ft_val_series.index = val_series.index
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    
    # pd.merge는 인덱스를 유지하지 않으므로 복원
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_val_series, noise_level), add_noise(ft_tst_series, noise_level)

In [11]:
train_df = pd.read_csv("C:/Users/daum0/Downloads/카카오톡 받은 파일/Kaggl study/2주차/Porto Seguro_train.csv", na_values="-1") # .iloc[0:200,:])
test_df = pd.read_csv("C:/Users/daum0/Downloads/카카오톡 받은 파일/Kaggl study/2주차/Porto Seguro_test.csv", na_values="-1")

In [12]:
# from olivier
train_features = [
    "ps_car_13",  #            : 1571.65 / shadow  609.23
	"ps_reg_03",  #            : 1408.42 / shadow  511.15
	"ps_ind_05_cat",  #        : 1387.87 / shadow   84.72
	"ps_ind_03",  #            : 1219.47 / shadow  230.55
	"ps_ind_15",  #            :  922.18 / shadow  242.00
	"ps_reg_02",  #            :  920.65 / shadow  267.50
	"ps_car_14",  #            :  798.48 / shadow  549.58
	"ps_car_12",  #            :  731.93 / shadow  293.62
	"ps_car_01_cat",  #        :  698.07 / shadow  178.72
	"ps_car_07_cat",  #        :  694.53 / shadow   36.35
	"ps_ind_17_bin",  #        :  620.77 / shadow   23.15
	"ps_car_03_cat",  #        :  611.73 / shadow   50.67
	"ps_reg_01",  #            :  598.60 / shadow  178.57
	"ps_car_15",  #            :  593.35 / shadow  226.43
	"ps_ind_01",  #            :  547.32 / shadow  154.58
	"ps_ind_16_bin",  #        :  475.37 / shadow   34.17
	"ps_ind_07_bin",  #        :  435.28 / shadow   28.92
	"ps_car_06_cat",  #        :  398.02 / shadow  212.43
	"ps_car_04_cat",  #        :  376.87 / shadow   76.98
	"ps_ind_06_bin",  #        :  370.97 / shadow   36.13
	"ps_car_09_cat",  #        :  214.12 / shadow   81.38
	"ps_car_02_cat",  #        :  203.03 / shadow   26.67
	"ps_ind_02_cat",  #        :  189.47 / shadow   65.68
	"ps_car_11",  #            :  173.28 / shadow   76.45
	"ps_car_05_cat",  #        :  172.75 / shadow   62.92
	"ps_calc_09",  #           :  169.13 / shadow  129.72
	"ps_calc_05",  #           :  148.83 / shadow  120.68
	"ps_ind_08_bin",  #        :  140.73 / shadow   27.63
	"ps_car_08_cat",  #        :  120.87 / shadow   28.82
	"ps_ind_09_bin",  #        :  113.92 / shadow   27.05
	"ps_ind_04_cat",  #        :  107.27 / shadow   37.43
	"ps_ind_18_bin",  #        :   77.42 / shadow   25.97
	"ps_ind_12_bin",  #        :   39.67 / shadow   15.52
	"ps_ind_14",  #            :   37.37 / shadow   16.65
]

# 조합을 추가

combs = [
    ('ps_reg_01', 'ps_car_02_cat'),  
    ('ps_reg_01', 'ps_car_04_cat'),
]

In [13]:
# 가공데이터
id_test = test_df['id'].values
id_train = train_df['id'].values
y = train_df['target']

start = time.time()
for n_c, (f1, f2) in enumerate(combs):
    name1 = f1 + "_plus_" + f2
    print('current feature %60s %4d in %5.1f'
          % (name1, n_c + 1, (time.time() - start) / 60), end='')
    print('\r' * 75, end='')
    train_df[name1] = train_df[f1].apply(lambda x: str(x)) + "_" + train_df[f2].apply(lambda x: str(x))
    test_df[name1] = test_df[f1].apply(lambda x: str(x)) + "_" + test_df[f2].apply(lambda x: str(x))
    
    # Label Encode
    lbl = LabelEncoder()
    lbl.fit(list(train_df[name1].values) + list(test_df[name1].values))
    train_df[name1] = lbl.transform(list(train_df[name1].values))
    test_df[name1] = lbl.transform(list(test_df[name1].values))

    train_features.append(name1)
    
X = train_df[train_features]
test_df = test_df[train_features]

f_cats = [f for f in X.columns if "_cat" in f]

current feature                                 ps_reg_01_plus_ps_car_04_cat    2 in   0.1

In [14]:
y_valid_pred = 0*y
y_test_pred = 0

In [15]:
# 접힘 설정
K = 5
kf = KFold(n_splits = K, random_state = 1, shuffle = True)
np.random.seed(0)

In [16]:
# 분류기 설정
model = XGBClassifier(    
                        n_estimators=MAX_ROUNDS,
                        max_depth=4,
                        objective="binary:logistic",
                        learning_rate=LEARNING_RATE, 
                        subsample=.8,
                        min_child_weight=6,
                        colsample_bytree=.8,
                        scale_pos_weight=1.6,
                        gamma=10,
                        reg_alpha=8,
                        reg_lambda=1.3,
                     )

In [17]:
# Run CV

for i, (train_index, test_index) in enumerate(kf.split(train_df)):
    
    # 이 접힘에 대한 데이터 만들기
    y_train, y_valid = y.iloc[train_index].copy(), y.iloc[test_index]
    X_train, X_valid = X.iloc[train_index,:].copy(), X.iloc[test_index,:].copy()
    X_test = test_df.copy()
    print( "\nFold ", i)
    
    # 암호화 데이터
    for f in f_cats:
        X_train[f + "_avg"], X_valid[f + "_avg"], X_test[f + "_avg"] = target_encode(
                                                        trn_series=X_train[f],
                                                        val_series=X_valid[f],
                                                        tst_series=X_test[f],
                                                        target=y_train,
                                                        min_samples_leaf=200,
                                                        smoothing=10,
                                                        noise_level=0
                                                        )
    # 모델 실행
    if OPTIMIZE_ROUNDS:
        eval_set=[(X_valid,y_valid)]
        fit_model = model.fit( X_train, y_train, 
                               eval_set=eval_set,
                               eval_metric=gini_xgb,
                               early_stopping_rounds=EARLY_STOPPING_ROUNDS,
                               verbose=False
                             )
        print( "  Best N trees = ", model.best_ntree_limit )
        print( "  Best gini = ", model.best_score )
    else:
        fit_model = model.fit( X_train, y_train )
        
    # 유효성 검사 예측 생성
    pred = fit_model.predict_proba(X_valid)[:,1]
    print( "  Gini = ", eval_gini(y_valid, pred) )
    y_valid_pred.iloc[test_index] = pred
    
    # 검정 집합 예측 누적
    y_test_pred += fit_model.predict_proba(X_test)[:,1]
    
    del X_test, X_train, X_valid, y_train
    
y_test_pred /= K  # 평균 검정 집합 예측

print( "\nGini for full training set:" )
eval_gini(y, y_valid_pred)


Fold  0


C:\Users\daum0\AppData\Local\Temp\ipykernel_8404\2717446381.py:4: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "eval_gini" failed type inference due to: non-precise type pyobject
During: typing of argument at C:\Users\daum0\AppData\Local\Temp\ipykernel_8404\2717446381.py (6)

File "..\..\..\..\AppData\Local\Temp\ipykernel_8404\2717446381.py", line 6:
<source missing, REPL/exec in use?>

  @jit
C:\Users\daum0\AppData\Local\Temp\ipykernel_8404\2717446381.py:4: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "eval_gini" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "..\..\..\..\AppData\Local\Temp\ipykernel_8404\2717446381.py", line 12:
<source missing, REPL/exec in use?>

  @jit
C:\Users\daum0\anaconda3\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function "eval_gini" was compiled in object mo

  Gini =  0.28244069442661

Fold  1
  Gini =  0.2758713611856801

Fold  2
  Gini =  0.26874747429256474

Fold  3
  Gini =  0.2977952308541023

Fold  4
  Gini =  0.2795135139545827

Gini for full training set:


C:\Users\daum0\AppData\Local\Temp\ipykernel_8404\2717446381.py:4: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "eval_gini" failed type inference due to: non-precise type pyobject
During: typing of argument at C:\Users\daum0\AppData\Local\Temp\ipykernel_8404\2717446381.py (6)

File "..\..\..\..\AppData\Local\Temp\ipykernel_8404\2717446381.py", line 6:
<source missing, REPL/exec in use?>

  @jit
C:\Users\daum0\AppData\Local\Temp\ipykernel_8404\2717446381.py:4: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "eval_gini" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "..\..\..\..\AppData\Local\Temp\ipykernel_8404\2717446381.py", line 12:
<source missing, REPL/exec in use?>

  @jit
C:\Users\daum0\anaconda3\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function "eval_gini" was compiled in object mo

0.2806146391268405

In [ ]:
# 적층/조립에 대한 유효성 검사 예측 저장
val = pd.DataFrame()
val['id'] = id_train
val['target'] = y_valid_pred.values
val.to_csv('xgb_valid.csv', float_format='%.6f', index=False)

In [ ]:
# 제출 파일 생성
sub = pd.DataFrame()
sub['id'] = id_test
sub['target'] = y_test_pred
sub.to_csv('xgb_submit.csv', float_format='%.6f', index=False)